# Single category investigation for 2 level classifier

This notebook is for investigation of one specific category at once.
Target data is employee's check data and flickr data.  
Model and other setup is based on prediction_2level_classifier.ipynb.

## 1. Set up

In [1]:
import os
import json
import glob
import scipy
import numpy as np

import pandas as pd
import glob

Using TensorFlow backend.


In [2]:
# set root as ../
import sys
sys.path.append("../")

In [3]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

In [4]:
BASE_CODE_NAME = "code_" + BASE_NAME

In [5]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

# Use this for get catkeys.
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [6]:
CATSTORE_PATH = ""

# Target data setup

In [7]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

In [8]:
EMPLOYEE_DATA_DIR = ''

In [10]:
EMPLOYEE_DATA_PATHS = list(glob.glob(os.path.normpath("{}/*.jpg").format(EMPLOYEE_DATA_DIR)))

In [11]:
FLICKR_DATA_DIR1 = ''
FLICKR_DATA_DIR2 = ''
FLICKR_DATA_SPAGHETTI=''

In [12]:
FLICKR_DATA_PATHS1 = list(glob.glob(os.path.normpath("{}/*.jpg").format(FLICKR_DATA_DIR1)))
FLICKR_DATA_PATHS2 = list(glob.glob(os.path.normpath("{}/*.jpg").format(FLICKR_DATA_DIR2)))
FLICKR_DATA_PATHS_SPAGHETTI = list(glob.glob(os.path.normpath("{}/*.jpg").format(FLICKR_DATA_SPAGHETTI)))


In [13]:
import random, itertools

In [14]:
# CHECK_DATA_PATHS = list(itertools.chain(random.sample(CHECK_DATA_PATHS1, 5000), CHECK_DATA_PATHS2))
FLICKR_DATA_PATHS = list(itertools.chain(FLICKR_DATA_PATHS1[0:5000], FLICKR_DATA_PATHS2[0:(5000-len(FLICKR_DATA_PATHS_SPAGHETTI))], FLICKR_DATA_PATHS_SPAGHETTI))

In [15]:
len(EMPLOYEE_DATA_PATHS), len(FLICKR_DATA_PATHS)

(1995, 10000)

### Specify which category you want to evaluate

In [18]:
targetcat = ()

### Classifier setup

In [30]:
targetkey = targetcat[0]

In [21]:
catproc = [targetcat]

In [23]:
h2binder = ModelBinder_create()
h1binder = ModelBinder.dup_from(h2binder)

In [24]:
from one_vs_all import TwoLevelClassifier

In [25]:
two = TwoLevelClassifier(catproc, h2binder, h1binder)

In [26]:
two.load_all()

load 93
load h1_93


### Evaluate threshold

Do inference and plot results

In [35]:
from visualize import plot_image_list

In [58]:
def plot_cat(df, origin=0):
    filtered = df
    for i in range(0, 50, 5):
        print(",".join(["{0:.4f}".format(v) for v in filtered[targetkey].values[origin+i:(origin+5+i)]]))
    plot_image_list(filtered["filepaths"].values[origin:(origin+50)])

In [59]:
def predict2nd(paths):
    # assume target cat is only one.
    key = two._catkeys[0]
    tup = two._categories[key]
    two._categories[key] = (tup[0], 0.5, tup[2])
    _ = two.predict_files_df(paths)
    df = two._df
    df['filepaths'] = paths
    df = df.sort_values(by=key, ascending=False)
    plot_cat(df)


In [63]:
# predict2nd(EMPLOYEE_DATA_PATHS)

### (try and error) Check what will returned in the end

In [27]:
def predict(paths, thrd1=0.99843749999999964, thrd2=0.97):
    # assume target cat is only one.
    key = two._catkeys[0]
    tup = two._categories[key]
    two._categories[key] = (thrd1, thrd2, tup[2])
    return two.predict_files_df(paths)

In [42]:
def eval_thresholds(paths, thrd1=0.99843749999999964, thrd2=0.97):
    df = predict(paths, thrd1, thrd2)
    pasta = df[df['category'] == targetkey]
    print((len(pasta), thrd1, thrd2))
    plot_image_list(pasta['filepaths'].values)

In [61]:
# eval_thresholds(EMPLOYEE_DATA_PATHS)

In [62]:
# eval_thresholds(EMPLOYEE_DATA_PATHS, thrd2=0.8)